In [19]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records
import models
models_path = models.__path__[0]
import os

fix_block_number = 41354245
fix_block_number = 41323403

fix_block_number = 41323403 - 1

fix_block_number = None

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 137,
}

context, _model_loader = create_cmf(cmf_param)

In [20]:
from models.credmark.protocols.ichi.vaults import ICHI_POLYGON_COINS, ICHI_POLYGON_VAULTS

In [21]:
if False:
    for coin in ICHI_POLYGON_COINS:
        tok = Token(coin).as_erc20(set_loaded=True)
        print((tok.symbol, tok.name, tok.decimals))

In [22]:
from models.tmp_abi_lookup import ICHI_VAULT, ICHI_VAULT_FACTORY, UNISWAP_V3_POOL_ABI

In [23]:
vault_factory = Contract('0x2d2c72C4dC71AA32D64e5142e336741131A73fc0').set_abi(ICHI_VAULT_FACTORY, set_loaded=True)
if False:
    vault_created = pd.DataFrame(vault_factory.fetch_events(vault_factory.events.ICHIVaultCreated, from_block=25_697_834, by_range=10_000))
    # vault_created = pd.DataFrame(vault_factory.fetch_events(vault_factory.events.ICHIVaultCreated, from_block=0))
    vault_created.to_pickle(os.path.join(models_path, '..', 'tmp', 'vault_created.pkl'))
else:
    vault_created = pd.read_pickle(os.path.join(models_path, '..', 'tmp', 'vault_created.pkl'))

# vault_created.ichiVault.isin(['0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81'])
# '0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81' in vault_created.ichiVault.to_list()

# '0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81' in ICHI_POLYGON_VAULTS
assert len(set(ICHI_POLYGON_VAULTS)) == len(set(vault_created.ichiVault))

# context.run_model('token.deployment', {'address': '0x2d2c72C4dC71AA32D64e5142e336741131A73fc0'})

In [24]:
if False:
    for n_vault, vault in enumerate(ICHI_POLYGON_VAULTS):
       vault = Contract(vault).set_abi(abi=ICHI_VAULT, set_loaded=True)
       token0 = Token(vault.functions.token0().call()).as_erc20(set_loaded=True)
       token1 = Token(vault.functions.token1().call()).as_erc20(set_loaded=True)

       print((n_vault, vault,
              vault.functions.owner().call(), vault.functions.pool().call(),
              token0.symbol,  token1.symbol,
              vault.functions.allowToken0().call(), vault.functions.allowToken1().call(),
              vault.functions.totalSupply().call() / (10**vault.functions.decimals().call())))
       
       # print((vault, vault.functions.pool().call()))

In [25]:
# allowToken0, allowToken1, balanceOf, getTotalAmounts, totalSupply, token0, token1, symbol, name41354245
# getBasePosition, getLimitPosition, hysteresis, 
# owner, ichiVaultFactory, pool, tickSpacing, fee

- vault operations
- withdraw, transfer
- deposit, transfer
- rebalance

- 0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7
- 0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0xED26b64F351A04Acbf16AA3782969e4a53B12c54', 'WETH', 'DPI', False, True, 280.73224104767195

all transfers

In [38]:
vault_addr = Address('0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7')
vault_ichi = Token(vault_addr).set_abi(abi=ICHI_VAULT, set_loaded=True)
vault_pool = Contract(Address(vault_ichi.functions.pool().call())).set_abi(UNISWAP_V3_POOL_ABI, set_loaded=True)

affiliate = Address(vault_ichi.functions.affiliate().call())
ichi_vault_factory = Contract(vault_ichi.functions.ichiVaultFactory().call()).set_abi(abi=ICHI_VAULT_FACTORY, set_loaded=True)
fee_recipient = Address(ichi_vault_factory.functions.feeRecipient().call())
baseFee = ichi_vault_factory.functions.baseFee().call()
baseFeeSplit = ichi_vault_factory.functions.baseFeeSplit().call()

token0 = Token(vault_ichi.functions.token0().call()).as_erc20(set_loaded=True)
token1 = Token(vault_ichi.functions.token1().call()).as_erc20(set_loaded=True)

allow_token0 = vault_ichi.functions.allowToken0().call()
allow_token1 = vault_ichi.functions.allowToken1().call()

(vault_addr, vault_pool, affiliate, fee_recipient, allow_token0, allow_token1, baseFee / 1e18, baseFeeSplit / 1e18)

('0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7',
 Contract(address='0xed26b64f351a04acbf16aa3782969e4a53b12c54'),
 '0x3d1e6349f508199fdcaf339c1c351dff1d5fd072',
 '0x0768a75f616b98ee0937673bd83b7abf142236ea',
 False,
 True,
 0.1,
 0.5)

In [27]:
context.block_number

41488883

In [28]:
(vault_ichi.functions.getTotalAmounts().call(),
 vault_ichi.functions.getBasePosition().call(),
 vault_ichi.functions.getLimitPosition().call(),
 token0.balance_of(vault_ichi.address.checksum),
 token1.balance_of(vault_ichi.address.checksum),
)

# 286852599603313080740 - 26182028192636748 - 286826417575120443992, \
# 274646900430582100 - 274646900430582100 - 0

# totalAmount0	totalAmount1	feeAmount0	feeAmount1	totalSupply
# 274911385738111039	286846445532138665343	16019734579489	0	280732241047671948180

([664150044500062852, 277987889939360362712],
 [3193493513008405666, 661920390290695023, 60054076162683351],
 [350987158883326741955, 2229654209367829, 277927835863197679361],
 0,
 0)

In [35]:
1000000000000000000

1.0

In [29]:
# vault_pool()
base_lower = vault_ichi.functions.baseLower().call()
base_upper = vault_ichi.functions.baseUpper().call()

limit_lower = vault_ichi.functions.limitLower().call()
limit_upper = vault_ichi.functions.limitUpper().call()

base_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [vault_ichi.address.checksum, base_lower, base_upper])
limit_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [vault_ichi.address.checksum, limit_lower, limit_upper])

vault_pool.functions.positions(base_position_key).call(), vault_pool.functions.positions(limit_position_key).call(), 

# liquidity uint128, feeGrowthInside0LastX128 uint256, feeGrowthInside1LastX128 uint256, tokensOwed0 uint128, tokensOwed1 uint128


([3193493513008405666,
  2875935123515081046214216637017803,
  98895976994737107529842726233571982,
  26999531736089,
  928444125045983],
 [350987158883326741955,
  2160902530510430322218937526472734,
  49766990538930166157750905769655732,
  2229654209367829,
  51350386413071495])

In [ ]:
# total supply
df_transfer = pd.DataFrame(vault_ichi.fetch_events(vault_ichi.events.Transfer, from_block=0))
df_transfer.loc[:, 'net_value'] = df_transfer.apply(lambda r: r.value if r['from'] == '0x0000000000000000000000000000000000000000' else -r.value, axis=1)
assert df_transfer.net_value.sum() == vault_ichi.total_supply
display(vault_ichi.total_supply_scaled)

In [ ]:
# deposit
depositor = Address('0x3fabe6f79f5119336886bbbdfc0b0853dc831919')

deposit_block = 41028728
context.web3.eth.default_block = deposit_block - 1
display((deposit_block - 1, vault_ichi.scaled(vault_ichi.balance_of(Address(depositor).checksum))))

context.web3.eth.default_block = deposit_block
display((deposit_block, vault_ichi.scaled(vault_ichi.balance_of(Address(depositor).checksum))))

# context.web3.eth.default_block = fix_block_number

In [ ]:
context.block_number

In [39]:
# fee
display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0, argument_filters={'from': vault_ichi.address.checksum, 'to': affiliate.checksum})))
display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0, argument_filters={'from': vault_ichi.address.checksum, 'to': fee_recipient.checksum})))

# TODO: deposit / withdraw by proportion.

# vault token value = (vault position value - fee owned by vault (10% of current in pool) / total Supply

# Flow: 
# Deposit - Withdraw - Rebalance 
# Deposit - Withdraw (In) / (vault position value - fee owned by vault (10% of current in pool)


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,value
0,"(from, to, value)",Transfer,412,87,b'P\xa8\x04~\x89\xc5J\x11=\x96\x19t\xe8$\x0e\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'R\x89\xb7\xcc\xb4\x8d\xed\x9c\x06A`l\xfdZ\x0...,39709682,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,146250557634778
1,"(from, to, value)",Transfer,35,10,b'\xaeI\xa6\xaa\x0bO!\xa8gJ\xca\x82FI\xe3\xc3\...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b' \n\xa9S\x0bP\x1b\tV8NA:Y\xd4Z8\x90u\xa9\xbf...,39740183,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,7151955731198
2,"(from, to, value)",Transfer,61,11,"b""\xf9'\x1d!\x01t\x01\xa5/\x02\x96\xd9\x92^\xc...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xb23\xb5k\xa2\x0b\xd3\x87\xaa\xd2a\xf1\xfac...,39773483,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,107033911253114
3,"(from, to, value)",Transfer,252,66,b'\x8dBz\x08<\x124\x03J\x960\x86\n\xfd\x88\x9d...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xd3W\x97\xce\x81kj\xad\xe0z\x88\xf4\xed\xf5...,39775221,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,887100513790587
4,"(from, to, value)",Transfer,187,66,"b""\x9b\xa0\n\xb7a\xbe\xd9\xdawK`H\x82\x93:Q\x8...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xa5|\xd2\tcwm\xdaAn4\x8bI\x92\xb3\xff\xa2\x...,39778761,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,455430885098485
...,...,...,...,...,...,...,...,...,...,...,...
62,"(from, to, value)",Transfer,64,15,b'\xcd_9\xdb\xe9\x90\xbf\\\xc2\xc8\x1d\x1f\xd8...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\n\x04\x90\xdf\xed\xfd\xfa\xbfsi\x88\xd8\xef...,41275026,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,1391213431769
63,"(from, to, value)",Transfer,50,8,b'\x92\x82\x02\x9c}\x1c\xac\x81\n\xafyQ\xd9{\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'N2X\x9c\xc4\x10\xe0S\x8a\xf5Le\xaf\x1e\x94W\...,41317592,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,23131681345805
64,"(from, to, value)",Transfer,145,7,"b""zb\xbf\x86\xd4Y\xd6C\x9a\xb5\xc7\xff`\xba\xd...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xdb7\x8b\xf4\x9f\xb0\x03\xb8\xc2\x9ec\x91I\...,41323403,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,800986728974
65,"(from, to, value)",Transfer,130,33,"b""\xd9=\xa2\xb8\x02\x91\xf9'\xb9\x1e\xbe\xd0\x...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\x7f\x9d\x8c\xe7]\xb8\xc9:\xff\xd2\xea\x8c\x...,41366445,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,197015049551019


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,value
0,"(from, to, value)",Transfer,411,87,b'P\xa8\x04~\x89\xc5J\x11=\x96\x19t\xe8$\x0e\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'R\x89\xb7\xcc\xb4\x8d\xed\x9c\x06A`l\xfdZ\x0...,39709682,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,146250557634777
1,"(from, to, value)",Transfer,34,10,b'\xaeI\xa6\xaa\x0bO!\xa8gJ\xca\x82FI\xe3\xc3\...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b' \n\xa9S\x0bP\x1b\tV8NA:Y\xd4Z8\x90u\xa9\xbf...,39740183,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,7151955731198
2,"(from, to, value)",Transfer,60,11,"b""\xf9'\x1d!\x01t\x01\xa5/\x02\x96\xd9\x92^\xc...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xb23\xb5k\xa2\x0b\xd3\x87\xaa\xd2a\xf1\xfac...,39773483,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,107033911253114
3,"(from, to, value)",Transfer,251,66,b'\x8dBz\x08<\x124\x03J\x960\x86\n\xfd\x88\x9d...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xd3W\x97\xce\x81kj\xad\xe0z\x88\xf4\xed\xf5...,39775221,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,887100513790587
4,"(from, to, value)",Transfer,186,66,"b""\x9b\xa0\n\xb7a\xbe\xd9\xdawK`H\x82\x93:Q\x8...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xa5|\xd2\tcwm\xdaAn4\x8bI\x92\xb3\xff\xa2\x...,39778761,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,455430885098485
...,...,...,...,...,...,...,...,...,...,...,...
62,"(from, to, value)",Transfer,63,15,b'\xcd_9\xdb\xe9\x90\xbf\\\xc2\xc8\x1d\x1f\xd8...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\n\x04\x90\xdf\xed\xfd\xfa\xbfsi\x88\xd8\xef...,41275026,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,1391213431769
63,"(from, to, value)",Transfer,49,8,b'\x92\x82\x02\x9c}\x1c\xac\x81\n\xafyQ\xd9{\x...,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'N2X\x9c\xc4\x10\xe0S\x8a\xf5Le\xaf\x1e\x94W\...,41317592,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,23131681345805
64,"(from, to, value)",Transfer,144,7,"b""zb\xbf\x86\xd4Y\xd6C\x9a\xb5\xc7\xff`\xba\xd...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\xdb7\x8b\xf4\x9f\xb0\x03\xb8\xc2\x9ec\x91I\...,41323403,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,800986728974
65,"(from, to, value)",Transfer,129,33,"b""\xd9=\xa2\xb8\x02\x91\xf9'\xb9\x1e\xbe\xd0\x...",0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,b'\x7f\x9d\x8c\xe7]\xb8\xc9:\xff\xd2\xea\x8c\x...,41366445,0x8AC3D7cd56816Da9fB45e7640aA70A24884e02f7,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,197015049551018


In [ ]:
# all deposit
# all withdraw
# all rebalance => fee


In [ ]:
for evt in vault_ichi.abi.events:
    if evt == 'Rebalance':
        df = pd.DataFrame(vault_ichi.fetch_events(vault_ichi.events[evt], from_block=0))
        print(evt)
        display(df)

In [ ]:
df.loc[107, 'transactionHash']